In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize
import time
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Functional Paradigm: Data downloading with retry mechanism
def download_data(tickers, start_date, end_date, retries=3, delay=5):
    for _ in range(retries):
        try:
            data = yf.download(tickers=tickers, start=start_date, end=end_date)
            if not data['Adj Close'].empty:
                return data['Adj Close']
        except Exception as e:
            print(f"Download failed: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
    print("Failed to download data. Switching to Excel backup...")
    return None

# Unified Class: StockPortfolio
class StockPortfolio:
    def __init__(self, tickers, start_date, end_date, excel_file, risk_free_rate=0.044):
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.excel_file = excel_file
        self.risk_free_rate = risk_free_rate
        self.prices = self.load_data()
        self.returns = np.log(self.prices / self.prices.shift(1)).dropna()
        self.pBar = self.returns.mean()
        self.Sigma = self.returns.cov()
        self.weights = np.array([0.1] * len(self.prices.columns))
        self.meanReturns, self.covMatrix = self.getData()

    def load_data(self):
        prices = download_data(self.tickers, self.start_date, self.end_date)
        if prices is None:
            prices = pd.read_excel(self.excel_file, index_col=0, parse_dates=True)
        return prices

    def getData(self):
        meanReturns = self.returns.mean()
        covMatrix = self.returns.cov()
        return meanReturns, covMatrix

    def calculate_metrics(self):
        port_variance = self.portfolio_variance(self.weights, self.Sigma)
        port_annual_ret = np.sum(self.pBar * self.weights)
        port_volatility = np.sqrt(port_variance)
        sharpe_ratio = (port_annual_ret - self.risk_free_rate) / port_volatility
        return port_annual_ret, port_volatility, port_variance, sharpe_ratio

    def portfolio_variance(self, weights, Sigma):
        return np.dot(weights.T, np.dot(Sigma, weights))

    def portfolioReturnsDaily(self, weights):
        returns = np.log(self.prices / self.prices.shift(1))
        return returns.dropna()
    
    def portfolioReturn(self, weights):  
        return np.sum(self.pBar * weights)

    def portfolioPerformance(self, weights):
        port_annual_ret = np.sum(self.meanReturns * weights)
        port_variance = self.portfolio_variance(weights, self.Sigma)
        port_volatility = np.sqrt(port_variance)
        return port_annual_ret, port_volatility

    def riskFunction(self, w):
        return self.portfolio_variance(w, self.Sigma)

    def singleEquationSolver(self):
        Sigma_inv = np.linalg.inv(self.Sigma)
        sum_all_elements = np.sum(Sigma_inv)
        w_opt = np.sum(Sigma_inv, axis=1) / sum_all_elements
        w_opt = np.maximum(w_opt, 0) / np.sum(np.maximum(w_opt, 0))
        return w_opt

    def markowitz_optimal_weights_specific_return(self, U):
        Sigma_inv = np.linalg.inv(self.Sigma)
        M = np.dot(np.dot(self.pBar.T, Sigma_inv), self.pBar)
        w_opt = np.dot(Sigma_inv, self.pBar) * (U / M)
        w_opt = np.maximum(w_opt, 0)
        return w_opt

    def allocation(self, method, U=None):
        if U is not None:
            optimized_weights = method(U)
        else:
            optimized_weights = method()

        # تحويل القيم إلى نسب مئوية
        optimized_allocation = pd.DataFrame(
            optimized_weights * 100,
            index=self.meanReturns.index,
            columns=["allocation"],
        )

        # تقريب القيم إلى خانتين عشريتين
        optimized_allocation["allocation"] = optimized_allocation["allocation"].round(2)

        # حساب المجموع الفعلي بعد التقريب
        total = optimized_allocation["allocation"].sum()

        # ضبط الفرق في آخر عنصر بحيث يصبح المجموع 100%
        if total != 100:
            diff = 100 - total
            max_idx = optimized_allocation["allocation"].idxmax()  # تحديد أكبر وزن
            optimized_allocation.at[max_idx, "allocation"] += diff  # ضبط الفرق عليه

        # تنسيق القيم كنسب مئوية
        optimized_allocation["allocation"] = optimized_allocation["allocation"].map(lambda x: f"{x:.2f}%")

        # إضافة صف المجموع
        total_row = pd.DataFrame({"allocation": ["100.00%"]}, index=["Total"])
        optimized_allocation = pd.concat([optimized_allocation, total_row])

        return optimized_allocation

    





# Integration and Execution
def main():
    tickers = ['AAPL', 'JNJ', 'PG', 'JPM', 'XOM', 'AMZN', 'KO', 'MSFT', 'GOLD', 'CVX']
    start_date = '2015-01-01'
    end_date = '2023-12-30'
    excel_file = "stock_data 2015-2023.xlsx"

    stock_portfolio = StockPortfolio(tickers, start_date, end_date, excel_file)

    port_annual_ret, port_volatility, port_variance, sharpe_ratio = stock_portfolio.calculate_metrics()
    print("\nInitial Portfolio Performance Metrics:")
    print(f"Expected Annual Return: {port_annual_ret * 100:.2f}%")
    print(f"Annual Volatility: {port_volatility * 100:.2f}%")
    print(f"Variance: {port_variance * 100:.2f}%")
    print(f"Sharpe Ratio: {sharpe_ratio:.3f}")

    w_opt_markowitz = stock_portfolio.singleEquationSolver()
    risk_markowitz = stock_portfolio.riskFunction(w_opt_markowitz)
    ret_markowitz = stock_portfolio.portfolioReturn(w_opt_markowitz)
    print("\nOptimized Markowitz Portfolio (Single Equation):")
    print(f"Weights: {w_opt_markowitz}")
    print(f"Expected Annual Return: {ret_markowitz:.2%}")
    print(f"Portfolio Risk (Variance): {risk_markowitz:.4f}")
    print(f"Sum of Weights: {np.sum(w_opt_markowitz):.4f}")

    U = 0.0005  # Target portfolio return

    # Calculate and display Markowitz portfolio with specific return target
    w_opt_specific_return = stock_portfolio.markowitz_optimal_weights_specific_return(U)
    risk_specific_return = stock_portfolio.riskFunction(w_opt_specific_return)
    ret_specific_return = stock_portfolio.portfolioReturn(w_opt_specific_return)
    print("\nOptimized specific return Portfolio:")
    print(f"Weights: {w_opt_specific_return}")
    print(f"Expected Annual Return: {ret_specific_return:.2%}")
    print(f"Portfolio Risk (Variance): {risk_specific_return:.4f}")
    print(f"Sum of Weights: {np.sum(w_opt_specific_return):.4f}")


    print("\nOptimized Allocation:")
    print(stock_portfolio.allocation(stock_portfolio.singleEquationSolver))

    print("\nOptimized Allocation:")
    print(stock_portfolio.allocation(stock_portfolio.markowitz_optimal_weights_specific_return, U))


if __name__ == "__main__":
    main()


[*********************100%***********************]  10 of 10 completed

10 Failed downloads:
['CVX', 'AAPL', 'GOLD', 'JNJ', 'PG', 'MSFT', 'JPM', 'XOM', 'KO', 'AMZN']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%***********************]  9 of 10 completed

10 Failed downloads:
['CVX', 'AAPL', 'GOLD', 'JNJ', 'PG', 'MSFT', 'JPM', 'XOM', 'KO', 'AMZN']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%***********************]  8 of 10 completed

10 Failed downloads:
['CVX', 'AAPL', 'GOLD', 'JNJ', 'PG', 'MSFT', 'JPM', 'XOM', 'KO', 'AMZN']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[**********************90%******************     ]  9 of 10 completed

Failed to download data. Switching to Excel backup...

Initial Portfolio Performance Metrics:
Expected Annual Return: 0.05%
Annual Volatility: 1.21%
Variance: 0.01%
Sharpe Ratio: -3.582

Optimized Markowitz Portfolio (Single Equation):
Weights: [0.00295601 0.31283279 0.21726903 0.         0.10489495 0.09151496
 0.26099396 0.         0.00953831 0.        ]
Expected Annual Return: 0.03%
Portfolio Risk (Variance): 0.0001
Sum of Weights: 1.0000

Optimized specific return Portfolio:
Weights: [0.12180903 0.         0.         0.10869028 0.         0.11125263
 0.         0.2323209  0.04060371 0.        ]
Expected Annual Return: 0.05%
Portfolio Risk (Variance): 0.0001
Sum of Weights: 0.6147

Optimized Allocation:
      allocation
AAPL       0.30%
JNJ       31.28%
PG        21.73%
JPM        0.00%
XOM       10.49%
AMZN       9.15%
KO        26.10%
MSFT       0.00%
TSLA       0.95%
CVX        0.00%
Total    100.00%

Optimized Allocation:
      allocation
AAPL      12.18%
JNJ        0.00%
PG     

In [6]:
import numpy as np

def main():
    tickers = ['AAPL', 'JNJ', 'PG', 'JPM', 'XOM', 'AMZN', 'KO', 'MSFT', 'GOLD', 'CVX']
    start_date = '2024-01-01'
    end_date = '2024-12-30'
    excel_file = "stock_data 2024.xlsx"

    stock_portfolio = StockPortfolio(tickers, start_date, end_date, excel_file)

    print("\n--- Portfolio Performance Comparison ---\n")

    # الأوزان الحالية (ماركوفيتز)
    w_opt_markowitz = np.array([0.00295601, 0.31283279, 0.21726903, 0.        , 0.10489495, 0.09151496,
                                0.26099396, 0.        , 0.00953831, 0.        ])
    ret_markowitz = stock_portfolio.portfolioReturn(w_opt_markowitz)
    risk_markowitz = stock_portfolio.riskFunction(w_opt_markowitz)

    print(f"🔹 Markowitz Portfolio Return: {ret_markowitz * 100:.2f}%")
    print(f"🔹 Markowitz Portfolio Risk (Variance): {risk_markowitz:.4f}")

    # الأوزان الجديدة (الحد الأدنى للعائد المحدد)
    w_opt_specific_return = np.array([0.12180903, 0.        , 0.        , 0.10869028, 0.        , 0.11125263,
                                        0.        , 0.2323209,  0.04060371, 0.        ])
    ret_specific_return = stock_portfolio.portfolioReturn(w_opt_specific_return)
    risk_specific_return = stock_portfolio.riskFunction(w_opt_specific_return)

    print("\nApplying New Weights...\n")
    print(f"🔹 Target Portfolio Return: {ret_specific_return * 100:.2f}%")
    print(f"🔹 Target Return Portfolio Risk (Variance): {risk_specific_return:.4f}")



if __name__ == "__main__":
    main()


[                       0%                       ]

[**********************80%*************          ]  8 of 10 completed

10 Failed downloads:
[**********************80%*************          ]  8 of 10 completed['CVX', 'AAPL', 'GOLD', 'JNJ', 'PG', 'MSFT', 'JPM', 'XOM', 'KO', 'AMZN']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%***********************]  10 of 10 completed

10 Failed downloads:
['CVX', 'AAPL', 'GOLD', 'JNJ', 'PG', 'MSFT', 'JPM', 'XOM', 'KO', 'AMZN']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%***********************]  8 of 10 completed

10 Failed downloads:
[**********************90%******************     ]  9 of 10 completed['CVX', 'AAPL', 'GOLD', 'JNJ', 'PG', 'MSFT', 'JPM', 'XOM', 'KO', 'AMZN']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


Failed to download data. Switching to Excel backup...

--- Portfolio Performance Comparison ---

🔹 Markowitz Portfolio Return: 0.02%
🔹 Markowitz Portfolio Risk (Variance): 0.0000

Applying New Weights...

🔹 Target Portfolio Return: 0.07%
🔹 Target Return Portfolio Risk (Variance): 0.0000
